# HW5: Transformer



This assignment will introduce you to

1. Understanding the structure of transformer.

2. Building a GPT model step by step

3. Train a GPT language model to write few sentences.

You can run this assignment on Colab or SCC. You are encouraged to use GPU to make training faster, but BE AWARE the Google Colab GPU limit: you are limited to use it for less than 12 hours continuously, after that you may not be able to access it for a particular duration of time unless you purchase Colab pro.



## Q1 Sequence to Sequence Modelling with nn.Transformer (20 points)

You will implement a part of transformer. This question aims to let you to get familiar with the transformer architecture purposed in the paper [Attention is all you need](https://arxiv.org/pdf/1706.03762.pdf). This question is modified from the original pytorch tutorial [here](https://pytorch.org/tutorials/beginner/transformer_tutorial.html?highlight=transformer), you can refer it when you fill out the code. The general architecture of trasnsformer is shown in the figure below:

<img src="https://pytorch.org/tutorials/_images/transformer_architecture.jpg" width="360em">

This question requires you to implement a sequence to sequence model by encoder, which is the left part of the figure. You will use integrated layers in pytorch.

The transformer model has been proved to be superior in quality for many sequence-to-sequence
problems while being more parallelizable. The ``nn.Transformer`` module
relies entirely on an attention mechanism (another module recently
implemented as `nn.MultiheadAttention`) to draw global dependencies
between input and output. The ``nn.Transformer`` module is now highly
modularized such that a single component (like [`nn.TransformerEncoder `](<https://pytorch.org/docs/master/nn.html?highlight=nn%20transformerencoder#torch.nn.TransformerEncoder>)
in this tutorial) can be easily adapted/composed.

### Q1.1 Define the model
In this question, we train ``nn.TransformerEncoder`` model on a
language modeling task. The language modeling task is to assign a
probability for the likelihood of a given word (or a sequence of words)
to follow a sequence of words. A sequence of tokens are passed to the embedding
layer first, followed by a positional encoding layer to account for the order
of the word (see the next paragraph for more details). The
``nn.TransformerEncoder`` consists of multiple layers of
``nn.TransformerEncoderLayer`` . Along with the input sequence, a square
attention mask is required because the self-attention layers in
``nn.TransformerEncoder`` are only allowed to attend the earlier positions in
the sequence. For the language modeling task, any tokens on the future
positions should be masked. To have the actual words, the output
of ``nn.TransformerEncoder`` model is sent to the final Linear
layer, which is followed by a log-Softmax function. We will see how to implement the ``PositionalEncoding`` in the later question.

<img src="https://www.researchgate.net/publication/334288604/figure/fig1/AS:778232232148992@1562556431066/The-Transformer-encoder-structure.ppm">




In the following model, we only train a encoder model, which is the left part of the figure. Then we concatenate a Linear model `self.decoder` to replace the right part of the model.

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):
    '''
    This is a transformer encoder model, the input arguments are as follows:
    args:
    ntoken:  dimension of tokens
    ninp: dimension of input embeddings
    nhid: dimension of the hidden encoding between two layers of TransformerEncoderLayer
    nlayers: number of TransformerEncoderLayer layers
    nhead: the number of heads in the multiheadattention model
    '''
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout) # PositionalEncoding will be implemented in next section.
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        """YOUR CODE HERE"""
        '''
        You can use torch.triu and masked_fill to get an upper triangle mask.
        The upper right entries are -inf, down left entries including the diagonal are 0.
        '''
        mask = torch.triu(torch.ones(sz, sz)) == 1
        mask = mask.transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

        return mask


    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        """YOUR CODE HERE"""
        '''
        Fill the forward function according to the diagram above.
        In the embedding layers, we multiply those weights by square root of
        self.ninp.
        '''
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask) if src_mask is not None else self.transformer_encoder(src)
        output = self.decoder(output)

        return output


### Q1.2 Positional Encoding
#### Q1.2.1 Fill the code block
``PositionalEncoding`` module injects some information about the
relative or absolute position of the tokens in the sequence. The
positional encodings have the same dimension as the embeddings so that
the two can be summed. Here, we use ``sine`` and ``cosine`` functions of
different frequencies.


In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]

        return self.dropout(x)

#### Q1.2.2 Why do we need this positional encoding in the transformer architectrue.

## Q2 Transformer Block for GPT (30 points)

### Q 2.1 Multi-head self-attention
#### Q 2.1.1 The first part is multi-head self-attention. In this layer, you will need to:
- Apply linear projections to convert the feature vector at each token into separate vectors for the query, key, and value. The input and output size of linear projection are both `n_embd`
- Apply attention, scaling the logits by $\frac{1}{\sqrt{d_{qkv}}}$.
- Ensure proper masking, such that padding tokens are never attended to.
- Perform attention `n_head` times in parallel, where the results are concatenated and then projected using a linear layer.

<img src="https://www.researchgate.net/publication/332139525/figure/fig3/AS:743081083158528@1554175744311/a-The-Transformer-model-architecture-b-left-Scaled-Dot-Product-Attention.ppm" width="360em">

You should include two types of dropout in your code (with probability set by the  `dropout` argument):
- Dropout should be applied to the output of the attention layer (just prior to the residual connection, denoted by "Add & Norm" in the first figure)
- Dropout should *also* be applied after the final projection.
Notes:
- Query, key, and value vectors should have shape `[batch_size, n_heads, sequence_len, d_qkv]`
- Apply a mask to the scaled dot product of Q and K, before the Softmax function. Let the entry to be a small enough number where the entry of the causal mask is zero. You can use `torch.tril` or `torch.triu` to create a mask, usually we define the mask as a lower triangular matrix. Lower left (incude the diagonal) entries are ones, rest of entries are zeros.
Then apply `tensor.masked_fill()` to the output of the scaled dot product of Q and K (It is also the input of softmax). Where the mask is zero, set the input to softmax to a negative number with very large magnitude.
- Attention logits and probabilities should have shape `[batch_size, n_heads, sequence_len, sequence_len]`
- Vaswani et al. define the output of the attention layer as concatenating the various heads and then multiplying by a matrix $W^O$. It's also possible to implement this is a sum without ever calling `torch.cat`: note that $\text{Concat}(head_1, \ldots, head_h)W^O = head_1 W^O_1 + \ldots + head_h W^O_h$ where $W^O = \begin{bmatrix} W^O_1\\ \vdots\\ W^O_h\end{bmatrix}$. You may define the `self.proj` this way.


In [4]:
import math
import logging

import torch
import torch.nn as nn
from torch.nn import functional as F
class MultiHeadSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    You can also use torch.nn.MultiheadAttention to validate your implementation

    """

    def __init__(self, n_embd, n_head, block_size, attn_pdrop=0.1, resid_pdrop=0.1):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        #Define key, query, value projections for all heads
        self.key = nn.Linear(n_embd, n_embd)
        self.query = nn.Linear(n_embd, n_embd)
        self.value = nn.Linear(n_embd, n_embd)

        self.proj = nn.Linear(n_embd, n_embd)
        self.attn_pdrop = nn.Dropout(attn_pdrop)
        self.resid_pdrop = nn.Dropout(resid_pdrop)

        mask = torch.tril(torch.ones(block_size, block_size))
        self.register_buffer("mask", mask.view(1, 1, block_size, block_size))

    def forward(self, x, layer_past=None):
        B, T, C = x.size() # B = Batch

        k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        score = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(k.size(-1))

        if layer_past is not None:
            past_key, past_value = layer_past
            k = torch.cat((past_key, k), dim=-2)
            v = torch.cat((past_value, v), dim=-2)

        if layer_past is None:
            score = score.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))

        attn = F.softmax(score, dim=-1)
        attn = self.attn_pdrop(attn)

        context = torch.matmul(attn, v)
        context = context.transpose(1, 2).contiguous().view(B, T, C)
        output = self.proj(context)
        output = self.resid_pdrop(output)

        return output


#### Q 2.1.2 Why do we need to divide a scale of the dot product of Q and K?

### Q2.2 Transformer
We will implement the transformer block, which is the blue box in the figure. You can use `nn.LayerNorm` layer to apply layer norm. We defined the feed forward layer as `self.mlp`.

Notice that where to use the layer norm is a design choice, you can change to see how it affect the final results in the application of Question 3.

<img src="https://www.researchgate.net/publication/358519951/figure/fig8/AS:1122134894092288@1644549215188/The-GPT-1-architecture-proposed-in-Radford-et-al-a-It-is-composed-of-12-stacked.ppm" width="240em">







In [5]:
class TransformerBlock(nn.Module):
    """ an Transformer block """
    def __init__(self, n_embd, n_head, block_size, attn_pdrop=0.1, resid_pdrop=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = MultiHeadSelfAttention(n_embd, n_head, block_size, attn_pdrop, resid_pdrop)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(resid_pdrop),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))

        return x



## Q3 GPT on Addition (30 points)
In this question, we will train an GPT transformer to do addition. We first need to get the dataset and encode addition equation to a vocabulary by integers since we want to use GPT dealing with sequences of integers, and completing them according to patterns in the data.

  The sum of two n-digit numbers gives a third up to (n+1)-digit number. So our
  encoding will simply be the n-digit first number, n-digit second number, and (n+1)-digit result, all simply concatenated together. Because each addition problem is so structured, there is no need to bother the model with encoding +, =, or other tokens. Each possible sequence has the same length, and simply contains the raw digits of the addition problem. As a few examples, the 2-digit problems:
- 85 + 50 = 135 becomes the sequence `[8, 5, 5, 0, 1, 3, 5]`
- 6 + 39 = 45 becomes the sequence `[0, 6, 3, 9, 0, 4, 5]`

We will also only train GPT on the final (n+1)-digits because the first two n-digits are always assumed to be given. So when we give GPT an exam later, we will e.g. feed it the sequence `[0, 6, 3, 9]`, which encodes that we'd like to add 6 + 39, and hope that the model completes the integer sequence with `[0, 4, 5]` in 3 sequential steps.

In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [7]:
from torch.utils.data import Dataset

class AdditionDataset(Dataset):
    """
    Define the addition dataset
    """

    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1

        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __getitem__(self, idx):
        # given a problem index idx, first recover the associated a + b
        idx = self.ixes[idx]
        nd = 10**self.ndigit
        a = idx // nd
        b = idx %  nd
        c = a + b
        render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028"
        dix = [int(s) for s in render] # convert each character to its token index
        # x will be input to GPT and y will be the associated expected outputs
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long) # predict the next token in the sequence
        y[:self.ndigit*2-1] = -100
        return x, y

In [8]:
# create a dataset for e.g. 2-digit addition
ndigit = 2
train_dataset = AdditionDataset(ndigit=ndigit, split='train')
test_dataset = AdditionDataset(ndigit=ndigit, split='test')
train_dataset[0] # sample a training instance just to see what one raw example looks like

(tensor([4, 7, 1, 7, 0, 6]), tensor([-100, -100, -100,    0,    6,    4]))

### Q3.1 Define the GPT model
Now, we start constructing the GPT model. As is shown in the figure, there are 12 transformer blocks concatenated together. In our model, we use `n_layer` to represent the number of blocks. In this question, you need to do the following:

- Define the `n_layer` transformer blocks `self.blocks`
- Fill out the forward function. Note that the positional embedding is not hard coded as the original transformer, it is learned during training.



<img src="https://www.researchgate.net/publication/358519951/figure/fig8/AS:1122134894092288@1644549215188/The-GPT-1-architecture-proposed-in-Radford-et-al-a-It-is-composed-of-12-stacked.ppm" width="240em">

In [13]:
class GPT(nn.Module):
    """  the full GPT language model, with a squence size of block_size """

    def __init__(self, vocab_size, n_embd, n_head, block_size, n_layer, embd_pdrop=0.1, attn_pdrop=0.1,resid_pdrop=0.1):
        super().__init__()

        # input embedding stem
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, block_size, n_embd))
        self.drop = nn.Dropout(embd_pdrop)
        # transformer
        self.blocks = nn.ModuleList([TransformerBlock(n_embd, n_head, block_size, attn_pdrop, resid_pdrop) for _ in range(n_layer)])

        # decoder head
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)

        self.block_size = block_size
        self.apply(self._init_weights)

        logger.info("number of parameters: %e", sum(p.numel() for p in self.parameters()))


    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def configure_optimizers(self, train_config):
        """
        You don't need to change this function. This is setting specific parameters for optimization.
        """

        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # special case the position embedding parameter in the root GPT module as not decayed
        no_decay.add('pos_emb')

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer

    def forward(self, x, targets=None):
        b, t = x.size()
        assert t <= self.block_size, "Cannot forward, model block size is exhausted."

        x = self.tok_emb(x)
        x = x + self.pos_emb[:, :t, :]
        x = self.drop(x)
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.head(x)

        return logits, None




###Q3.2 Training the model

##### You will train the GPT model. Fill out the code of the training process.

In [14]:
import math
import logging
from tqdm import tqdm
import numpy as np
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data.dataloader import DataLoader

Setting some parameters for training. Initialize the GPT model.

In [15]:
logger = logging.getLogger(__name__)
class TrainerConfig:
    # optimization parameters
    max_epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1 # only applied on matmul weights
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False
    warmup_tokens = 375e6 # these two numbers come from the GPT-3 paper, but may not be good defaults elsewhere
    final_tokens = 260e9 # (at what point we reach 10% of original LR)
    # checkpoint settings
    ckpt_path = None
    num_workers = 0 # for DataLoader

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)
# initialize a baby GPT model
model = GPT(vocab_size = train_dataset.vocab_size, n_embd=128, n_head=4, block_size =  train_dataset.block_size, n_layer=2)

You need to fill out training process.
- Forward the model with current batch `x`, `y`;
- Zero the grad before update;
- Backward the loss and update the model parameter;
- You might want to use `torch.nn.utils.clip_grad_norm_`. The parameter max_norm is `config.grad_norm_clip`;
- You will run this getting a loss around 0.1 and accuracy on both train and test around 99%.

#### 3.2.1 Why do we need to use `torch.nn.utils.clip_grad_norm_` in training?

In [16]:
config = TrainerConfig(max_epochs=50, batch_size=512, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(train_dataset)*(ndigit+1),
                      num_workers=4)

device = 'cpu'
if torch.cuda.is_available():
  device = torch.cuda.current_device()
  model.to(device)
optimizer = model.configure_optimizers(config)



tokens = 0
for epoch in range(config.max_epochs):
    model.train()
    data = train_dataset
    loader = DataLoader(data, shuffle=True, pin_memory=True,
                        batch_size=config.batch_size,
                        num_workers=config.num_workers)
    losses = []
    pbar = tqdm(enumerate(loader), total=len(loader))
    for iter, (x, y) in pbar:
        # place data on the correct device
        x = x.to(device)
        y = y.to(device)

        logits, _ = model(x)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
        optimizer.step()

        # decay the learning rate based on our progress
        if config.lr_decay:
            tokens += (y >= 0).sum() # number of tokens processed this step (i.e. label is not -100)
            if tokens < config.warmup_tokens:
                # linear warmup
                lr_mult = float(tokens) / float(max(1, config.warmup_tokens))
            else:
                # cosine learning rate decay
                progress = float(tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
            lr = config.learning_rate * lr_mult
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        else:
            lr = config.learning_rate
        # report progress
        pbar.set_description(f"epoch {epoch+1} iter {iter}: train loss {loss.item():.5f}. lr {lr:e}")


epoch 1 iter 17: train loss 1.76087. lr 5.994512e-04: 100%|██████████| 18/18 [00:04<00:00,  3.96it/s]
epoch 2 iter 17: train loss 1.55655. lr 5.977197e-04: 100%|██████████| 18/18 [00:04<00:00,  3.67it/s]
epoch 3 iter 17: train loss 1.36094. lr 5.948114e-04: 100%|██████████| 18/18 [00:05<00:00,  3.49it/s]
epoch 4 iter 17: train loss 1.25314. lr 5.907379e-04: 100%|██████████| 18/18 [00:04<00:00,  4.06it/s]
epoch 5 iter 17: train loss 1.19641. lr 5.855153e-04: 100%|██████████| 18/18 [00:05<00:00,  3.31it/s]
epoch 6 iter 17: train loss 1.12037. lr 5.791641e-04: 100%|██████████| 18/18 [00:05<00:00,  3.31it/s]
epoch 7 iter 17: train loss 1.09238. lr 5.717095e-04: 100%|██████████| 18/18 [00:04<00:00,  4.07it/s]
epoch 8 iter 17: train loss 1.07742. lr 5.631810e-04: 100%|██████████| 18/18 [00:05<00:00,  3.09it/s]
epoch 9 iter 17: train loss 1.03604. lr 5.536122e-04: 100%|██████████| 18/18 [00:04<00:00,  3.97it/s]
epoch 10 iter 17: train loss 0.92611. lr 5.430411e-04: 100%|██████████| 18/18 [00:

Now you can run the following code to test the training data sne testing data. You should reach more than 95% correctness on both train and testing data.

In [17]:
def top_k_logits(logits, k):
    v, ix = torch.topk(logits, k)
    out = logits.clone()
    out[out < v[:, [-1]]] = -float('Inf')
    return out

def sample(model, x, steps, temperature=1.0, sample=False, top_k=None):
    """
    take a conditioning sequence of indices in x (of shape (b,t)) and predict the next token in
    the sequence, feeding the predictions back into the model each time.
    """
    block_size = train_dataset.block_size
    model.eval()
    for k in range(steps):
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
        logits, _ = model(x_cond)
        # pluck the logits at the final step and scale by temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop probabilities to only the top k options
        if top_k is not None:
            logits = top_k_logits(logits, top_k)
        # apply softmax to convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution or take the most likely
        if sample:
            ix = torch.multinomial(probs, num_samples=1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        # append to the sequence and continue
        x = torch.cat((x, ix), dim=1)
    return x
def Addition_GPT(dataset, batch_size=32, max_batches=-1):

    results = []
    loader = DataLoader(dataset, batch_size=batch_size)
    for b, (x, y) in enumerate(loader):
        x = x.to(device)
        d1d2 = x[:, :ndigit*2]
        d1d2d3 = sample(model, d1d2, ndigit+1)
        d3 = d1d2d3[:, -(ndigit+1):]
        factors = torch.tensor([[10**i for i in range(ndigit+1)][::-1]]).to(device)
        # decode the integers from individual digits
        d1i = (d1d2[:,:ndigit] * factors[:,1:]).sum(1)
        d2i = (d1d2[:,ndigit:ndigit*2] * factors[:,1:]).sum(1)
        d3i_pred = (d3 * factors).sum(1)
        d3i_gt = d1i + d2i
        correct = (d3i_pred == d3i_gt).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line, lol
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            judge = 'CORRECT' if correct[i] else 'WRONG'
            if not correct[i]:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)"
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))

        if max_batches >= 0 and b+1 >= max_batches:
            break

    print("final score: %d/%d = %.2f%% correct" % (np.sum(results), len(results), 100*np.mean(results)))

In [18]:
Addition_GPT(train_dataset, batch_size=1024, max_batches=10)

GPT claims that 009 + 000 = 019 (gt is 009; WRONG)
GPT claims that 045 + 055 = 090 (gt is 100; WRONG)
final score: 8998/9000 = 99.98% correct


In [19]:
Addition_GPT(test_dataset, batch_size=1024, max_batches=10)

GPT claims that 055 + 045 = 090 (gt is 100; WRONG)
final score: 999/1000 = 99.90% correct





## Q4 minGPT Text Completion(20 points)


In this question, we will train a GPT to write something fun. First, we need to download some Harry Potter novels as the training dataset. Then a `CharDataset` class is provided to help you prepare training data in the format of characters

In [20]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%202%20-%20The%20Chamber%20Of%20Secrets.txt"
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%203%20-%20Prisoner%20of%20Azkaban.txt"


--2023-12-06 22:52:48--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  --.-KB/s    in 0.04s   

2023-12-06 22:52:49 (11.8 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]

--2023-12-06 22:52:49--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%202%20-%20The%20Chamber%20Of%20Secrets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.

In [21]:

import math
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from collections.abc import Iterable
from torch.optim import Optimizer
from torch.utils.data import Dataset

In [22]:
class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = list(set(data))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))

        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data

    def __len__(self):
        return math.ceil(len(self.data) / (self.block_size + 1))

    def __getitem__(self, idx):
        # we're actually going to "cheat" and pick a spot in the dataset at random
        i = np.random.randint(0, len(self.data) - (self.block_size + 1))
        chunk = self.data[i:i+self.block_size+1]
        dix = [self.stoi[s] for s in chunk]
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [23]:
# the "block size" is the number of characters the model takes as input.
# in this case, it can look at up to 128 characters when predicting the next
# character.
block_size = 128 # spatial extent of the model for its context

# For our training set, we will use the text of the first three Harry Potter books.
text = open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'rb').read()
text += open('J. K. Rowling - Harry Potter 2 - The Chamber Of Secrets.txt', 'rb').read()
text += open('J. K. Rowling - Harry Potter 3 - Prisoner of Azkaban.txt', 'rb').read()

train_dataset = CharDataset(text, block_size)

data has 1543473 characters, 87 unique.


### Q4.1 Train the model on texts and Save the model

Here we are going to train a language model on the `train_dataset`. In this question, you need to finish writing a training loop and save the best model (either the loss is small or the model output is fun) you get. You can refer to the training loop from Q3, but remember to save the model checkpoint(s) while training or at the end of the training loop - You will need it in the next question!



In the next cell, an example setup and parameters for the minGPT language model is provided, feel free to change the parameters. If you encounter a training loss plateau - loss is oscillating around some number and the tendency of decreasing is not obvious - and cannot overcome it when trying different parameter combinations,
- document the parameters in `TrainerConfig`, training loss curves/loss from the best model you have , the machine you used to train the model and
- propose a solution to overcome this plateau  

If you there is no such plateau, tell us what are the parameters in the `TrainConfig` and which GPU is used for training.

In this question, you are requried to
- write a training loop that saves some checkpoints in the training loop
- show some discussion on the training loss plateau

In [24]:
config = TrainerConfig(max_epochs=200, batch_size=64, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=150*len(train_dataset),
                      num_workers=4)

model = GPT(vocab_size=train_dataset.vocab_size,n_embd=128, n_head=8,block_size=train_dataset.block_size,n_layer=12,)
device = 'cpu'
if torch.cuda.is_available():
  device = torch.cuda.current_device()
  model.to(device)
  optimizer = model.configure_optimizers(config)


In [25]:
import os
import torch
import numpy as np


config = TrainerConfig(
    max_epochs=200,
    batch_size=64,
    learning_rate=6e-4,
    lr_decay=True,
    warmup_tokens=1024,
    final_tokens=150 * len(train_dataset),
    num_workers=4
)


model = GPT(
    vocab_size=train_dataset.vocab_size,
    n_embd=128,
    n_head=8,
    block_size=train_dataset.block_size,
    n_layer=12,
)


device = 'cpu'
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    model.to(device)
    optimizer = model.configure_optimizers(config)


best_loss = float('inf')
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(config.max_epochs):
    model.train()
    data = train_dataset
    loader = DataLoader(data, shuffle=True, pin_memory=True,
                        batch_size=config.batch_size,
                        num_workers=config.num_workers)

    losses = []
    pbar = tqdm(enumerate(loader), total=len(loader))

    for iter, (x, y) in pbar:
        x = x.to(device)
        y = y.to(device)

        logits, _ = model(x)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
        optimizer.step()

        if config.lr_decay:
            tokens += (y >= 0).sum()
            if tokens < config.warmup_tokens:
                lr_mult = float(tokens) / float(max(1, config.warmup_tokens))
            else:
                progress = float(tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
            lr = config.learning_rate * lr_mult
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        pbar.set_description(f"epoch {epoch+1} iter {iter}: train loss {loss.item():.5f}. lr {lr:e}")

    avg_loss = sum(losses) / len(losses)

    if avg_loss < best_loss:
        best_loss = avg_loss
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Saved checkpoint at epoch {epoch + 1} with loss {best_loss:.5f}")


final_checkpoint_path = os.path.join(checkpoint_dir, 'final_model.pt')
torch.save(model.state_dict(), final_checkpoint_path)
print(f"Training completed. Final model saved at {final_checkpoint_path}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
epoch 1 iter 186: train loss 4.49314. lr 3.979532e-04: 100%|██████████| 187/187 [21:39<00:00,  6.95s/it]

Saved checkpoint at epoch 1 with loss 4.49125



epoch 2 iter 186: train loss 4.49576. lr 3.378866e-04: 100%|██████████| 187/187 [20:05<00:00,  6.45s/it]

Saved checkpoint at epoch 2 with loss 4.49115



epoch 3 iter 186: train loss 4.49595. lr 1.341243e-04: 100%|██████████| 187/187 [19:39<00:00,  6.31s/it]

...

Saved checkpoint at epoch 6 with loss 4.49052



epoch 7 iter 186: train loss 4.48542. lr 1.024592e-04: 100%|██████████| 187/187 [19:31<00:00,  6.26s/it]
epoch 8 iter 186: train loss 4.48940. lr 3.769928e-04: 100%|██████████| 187/187 [18:11<00:00,  5.84s/it]
epoch 9 iter 186: train loss 4.48803. lr 3.595096e-04: 100%|██████████| 187/187 [18:49<00:00,  6.04s/it]
epoch 10 iter 186: train loss 4.49409. lr 1.163195e-04: 100%|██████████| 187/187 [18:58<00:00,  6.09s/it]

Saved checkpoint at epoch 10 with loss 4.49050



epoch 11 iter 5: train loss 4.48339. lr 1.373938e-04:   3%|▎         | 6/187 [00:44<22:09,  7.35s/it]


KeyboardInterrupt: ignored

Discussion

A situation where the training loss stabilizes without notable improvement, known as a training loss plateau, can be tackled by modifying the learning rate schedule, with the option of more aggressive decay when necessary. Trying out smaller model sizes might result in quicker convergence, especially for simpler tasks such as digit addition. To prevent overfitting, one can implement early stopping by keeping an eye on validation loss. Introducing data augmentation techniques can introduce diversity into the training data, potentially facilitating convergence. Finally, it is essential to engage in hyperparameter tuning, which encompasses batch size, model architecture, and optimizer settings, in order to discover the optimal combination for the given task.


It is used to address challenges in the training process of machine learning models. It emphasizes several key aspects:

Training Loss Plateau: The issue of a training loss plateau is acknowledged, and the solution involves adjusting the learning rate schedule, with a suggestion for more aggressive decay when necessary. This reflects an understanding of the importance of optimizing the learning process for better model performance.

Model Size and Convergence: The paragraph suggests that experimenting with smaller model sizes may lead to faster convergence, especially for simpler tasks like digit addition. This reflects an awareness of the relationship between model complexity and task difficulty.

Overfitting Prevention: To prevent overfitting, the paragraph recommends employing early stopping based on monitoring validation loss. This indicates an understanding of the need to balance model complexity and generalization to unseen data.

Data Augmentation: The introduction of data augmentation techniques is proposed to diversify the training data, potentially aiding convergence. This shows an appreciation for the impact of data quality and diversity on model training.

Hyperparameter Tuning: The paragraph highlights the significance of hyperparameter tuning, including batch size, model architecture, and optimizer settings. This reflects an understanding that the effectiveness of a machine learning model is influenced by various configuration parameters.

### Q4.2 Load pretrain model and do a prompt writing

Load the best model from Q4.1, provide a prompt and let the model continue writing for you. Feel free to try different prompts or different models.  
If your best model is trained on SCC or the other machines, load the model in the jupyter notebook you are going to submit and print out the 'writing'. Screenshot of the output 'writing' will not be accepted.

In this question, you are required to
- show the prompt writing output
- show some discussion for 4.2.1

#### 4.2.1 What would you do to improve the text generation quality(readability, spelling, grammar, logic etc.) of transformer-based language model? If you refer to some papers or posts, remember to cite them.

To enhance the quality of generated text, various methods can be utilized. One approach is data augmentation, which involves integrating additional data sources or domains during training and fine-tuning to augment the model's capacity and performance. Another strategy is model optimization, which involves adjusting the architecture and parameters to improve the overall capacity and performance of the model. After text generation, techniques for text refinement are applied to correct errors, including those related to grammar, spelling, punctuation, and style. The use of fine-tuned pretrained language models, achieved through transfer learning, is effective in generating coherent and contextually relevant text. Fine-tuning allows pretrained models to adapt to specific domains or tasks, enhancing the relevance and accuracy of the generated text. Additionally, adversarial training can be employed to improve the language model's robustness during training, enabling it to handle textual manipulations and produce more accurate and contextually appropriate responses. These strategies collectively contribute to the enhancement of text generation quality.

Various approaches are used to improve the quality of text generation. These approaches include data augmentation, model optimization, text refinement techniques, leveraging fine-tuned pretrained language models through transfer learning, and employing adversarial training. The goal is to address challenges related to coherence, context, and accuracy in generated text. Each strategy plays a distinct role, collectively contributing to the overall improvement of text generation quality.

In [26]:
# Example output
# model architecture should be defined before you load the parameters
PATH='/content/checkpoints/model_epoch_10.pt'
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [27]:
# Example output
prompt ="Harry Potter turned on the TV,  "
context=[ord(c) for c in prompt]
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(device)
y = sample(model, x, 200, temperature=0.6, sample=True, top_k=5)[0]
completion = ''.join([chr(train_dataset.itos[int(i)]) for i in y])
print(completion)

Harry Potter turned on the TV,  IOz":EE"1PHShI"1HShHpEEhHpEz":EhIOP3I"HP1hIO":Ez"SzHzzwz"H"SPzwHPHEH"SHEz"H"SHEzShI"HS"H""oIEHzPhIHSHvE"hooEhHhHSP":"n":Igpgg""vg!"0IYg!Qh":ng""noo""Sp!!g!"hgp!!3gp"oghoB/oIOOPng""""oo""oo"""nh""oIOwB
